# Simple _**agentic**_ RAG demo

Or more precisely: **CRAG** (_Corrective RAG_) - a technique for a little bit smarter RAG

<a target="_blank" href="https://colab.research.google.com/github/tolo/simple-rag-agent-demo/blob/main/simple-rag-agent-demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a><br/>

![Corrective RAG](https://github.com/tolo/simple-rag-agent-demo/blob/main/images/crag-flow.png?raw=true)

## Setup

### Install dependencies

In [ ]:
%pip install httpx~=0.28.1 openai~=1.57 --upgrade --quiet
%pip install python-dotenv~=1.0 docarray~=0.40.0 pypdf~=5.1 --upgrade --quiet
%pip install chromadb~=0.5.18 lark~=1.2 --upgrade --quiet
%pip install langchain~=0.3.10 langchain_openai~=0.2.11 langchain_community~=0.3.10 langchain-chroma~=0.1.4 --upgrade --quiet
%pip install langgraph~=0.2.56 --upgrade --quiet

# If running locally, you can do this instead:
#%pip install -r ../requirements.txt

### Load environment variables

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# If running in Google Colab, you can use this code instead:
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

### Setup Chat Model

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
llm = ChatOpenAI(model_name="gpt-4o-mini",temperature=0.0)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

## Setup ingestion / retrieval pipeline

### Setup vector DB (Chroma)

In [ ]:
from langchain_chroma import Chroma

persist_directory = './db/chroma/'
vectordb: Chroma = Chroma(
    collection_name="my_index",
    embedding_function=embedding_model,
    persist_directory=persist_directory # Optionally persist the database
)

retriever = vectordb.as_retriever()

### Setup a text splitter

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 80
)

### Setup documents to load

In [ ]:
# Documents to load (tuple of document_id and document_url)
from dataclasses import dataclass

@dataclass
class DocInfo:
    id: str
    url: str

documents: list[DocInfo] = [
    DocInfo("1", "https://data.riksdagen.se/fil/B9E2F955-31EA-4E9E-91EB-9AE0A3A8FFA7"), # Förordning om artificiell intelligens, 2020/21:FPM109
    DocInfo("2", "https://data.riksdagen.se/fil/BECC9F0F-3DA1-4F44-9417-02DF027DA29C"), # VITBOK Om artificiell intelligens - en EU-strategi för spetskompetens och förtroende
    DocInfo("3", "https://data.riksdagen.se/fil/C40BB689-7E23-4593-BDC6-DBEE327C00C6"), # Risker och möjligheter med artificiell intelligens, 2022/23:374
    DocInfo("4", "https://data.riksdagen.se/fil/4C47740C-D13E-4E22-80CB-43DD1E101080"), # Direktiv om skadeståndsansvar gällande artificiell intelligens, 2022/23:FPM8
]

### Ingest - split and add to vector index

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
import time

def ingest_documents(doc_info: DocInfo):
    # Check if document already exists
    existing = vectordb.get(where={"doc_id": doc_info.id})
    if existing["documents"]:
        print(f"Document {doc_info.id} already exists in index")
        return

    # Load
    print(f"Loading document {doc_info.id} ({doc_info.url})...")
    loader = PyPDFLoader(doc_info.url)
    pages = loader.load()
    for page in pages:
        page.metadata["doc_id"] = doc_info.id

    # Split
    doc_splits = text_splitter.split_documents(pages)

    # Add to index
    print(f"Adding document {doc_info.id} ({doc_info.url}) to index...")

    # Add in batches, with delay, to avoid rate limiting
    batch_size = 10
    for i in range(0, len(doc_splits), batch_size):
        batch = doc_splits[i:i + batch_size]
        vectordb.add_documents(documents=batch)
        print(f"Added splits {i} to {i + batch_size}")
        time.sleep(0.1)

    print(f"Added document {doc_info.id} ({doc_info.url}) ({len(pages)} pages) - {len(doc_splits)} splits")


for doc_info in documents:
    ingest_documents(doc_info)

## Setup query graph / pipeline

### Graph state

In [ ]:
from typing import  List

from langchain_core.documents import Document
from langgraph.graph import MessagesState


class GraphState(MessagesState):
    question: str
    documents: List[Document]
    irrelevant_docs: bool
    answer: str


### Nodes

#### Retrieval (Vector Store similarity search)

In [ ]:
class RetrievalNode:
    def __call__(self, state: GraphState):
        print("---RETRIEVE---")
        question = state["question"]

        # Retrieval
        documents = retriever.invoke(question)

        print(f"---RETRIEVED {len(documents)} DOCS---")
        #print(f"{documents}")

        return {**state, "documents": documents}
        #return {"question": question, "documents": documents}

#### Retrieval Grader

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable
from langchain_core.output_parsers import StrOutputParser

class RetrievalGraderNode:
    system_template = """You are a grader assessing relevance of a retrieved document to a user question.
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.
    Give a binary score '1' or '0' score to indicate whether the document is relevant to the question.

    **Retrieved document:** \n\n {document}
    """

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_template),
            ("human", "{question}"),
        ]
    )

    chain: Runnable

    def __init__(self):
        self.chain = self.prompt | llm | StrOutputParser()

    def __call__(self, state: GraphState):
        print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
        question = state["question"]
        documents = state["documents"]

        # Score each doc
        filtered_docs = []
        irrelevant_docs = True

        for (i, d) in enumerate(documents):
            grade = self.chain.invoke(
                {"question": question, "document": d.page_content}
            )
            if "1" in grade:
                print(f"---GRADE: DOCUMENT {i} RELEVANT---")
                filtered_docs.append(d)
                irrelevant_docs = False
            else:
                print(f"---GRADE: DOCUMENT {i} NOT RELEVANT---")
                continue

        return {**state, "documents": filtered_docs, "irrelevant_docs": irrelevant_docs}

#### Web Search (in case irrelevnat relevant docs were found)

As a fallback when there are no/few relevant docs, we can use a web search tool to find more information. In this case, we use the service [Tavily](https://tavily.com).

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

class WebSearchNode:
    web_search_tool: TavilySearchResults

    def __init__(self):
        self.web_search_tool = TavilySearchResults(max_results=8)

    def __call__(self, state: GraphState):
        print("---WEB SEARCH---")
        question = state["question"]
        documents = state["documents"]

        docs = self.web_search_tool.invoke({"query": question})
        web_results_urls = "\n".join([d["url"] for d in docs])
        print("---WEB SEARCH RESULT:")
        print(web_results_urls)

        web_results = "\n".join([d["content"] for d in docs])
        web_results = Document(page_content=web_results)
        documents.append(web_results)

        return {"documents": documents, "question": question}


# class FakeWebSearchNode:
#     system_template = """You are a helpful and cheerful assistant."""
#
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             ("system", system_template),
#             ("human", "{question}"),
#         ]
#     )
#
#     chain: Runnable
#
#     def __init__(self, llm):
#         self.chain = self.prompt | llm.bind(temperature=1.0) | StrOutputParser()
#
#     def __call__(self, state: GraphState):
#         print("---FAKE WEB SEARCH---")
#         question = state["question"]
#
#         web_results = self.chain.invoke({"question": question})
#
#         print(f"---FAKE WEB SEARCH RESULT: \n{web_results}")
#
#         web_results = Document(page_content=web_results)
#         documents.append(web_results)
#
#         return {"documents": documents, "question": question}

#### RAG Generation (LLM call with factual/grounded context)

In [ ]:
class RAGNode:
    system_template = """You are an helpful assistant, expert in answering questions based on provided sources (snippets from documents) and citing the sources used to generate the answer. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible.
    ALWAYS respond in the SAME language as the original question.

    ** Context (snippets from documents): **

    {context}
    """

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_template),
            ("human", "{question}"),
        ]
    )

    chain: Runnable

    def __init__(self):
        self.chain = self.prompt | llm | StrOutputParser()

    def __call__(self, state: GraphState):
        print("---GENERATE---")
        question = state["question"]
        documents = state["documents"]

        # RAG generation - setup context (i.e. relevant documents snippets)
        context = "\n\n".join(doc.page_content for doc in documents)

        # RAG generation - generate answer
        answer = self.chain.invoke({"question": question, "context": context})
        #print(f"---GENERATE - ANSWER: \n{answer}")

        return {"documents": documents, "answer": answer}

### Conditional edges

In [ ]:
def decide_to_generate(state):
    print("---ASSESS GRADED DOCUMENTS---")
    irrelevant_docs: bool = state["irrelevant_docs"]

    if irrelevant_docs:
        print(
            "---DECISION: USE WEB SEARCH---"
        )
        return "fallback"
    else:
        print("---DECISION: GENERATE---")
        return "generate"

### Build Graph

In [ ]:
#### Graph ####
from langgraph.graph import END, StateGraph, START
from IPython.display import Image, display

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", RetrievalNode())  # retrieve
workflow.add_node("grade_documents", RetrievalGraderNode())  # grade documents
workflow.add_node("web_search", WebSearchNode())  # fallback - web search
workflow.add_node("generate", RAGNode())  # generate

workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")

workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "generate": "generate",
        "fallback": "web_search",
    },
)

workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

#memory = MemorySaver()

# Compile
graph = workflow.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))


## Use Graph

In [ ]:
# Run
inputs = {
    "question": "Vad innebär vitboken om artificiell intelligens?" # Should NOT result in web search
    #"question": "Vilka är nobelpristagarna 2024?" # Should result in web search
    #"question": "Har det i riksdagen diskuterats något om risker kring användningen av artificiell intelligens (AI)?"
}

config = {
    "thread_id": "1",
    #"max_concurrency": 1,
    #"recursion_limit": 5
}

# Execute graph
result = graph.invoke(inputs, {"configurable": config})

print(f"--- ANSWER: ---\n{result['answer']}")